In [1]:
import pandas as pd
import plotly
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import nltk
stop=nltk.corpus.stopwords.words('english')
from nltk import word_tokenize
import stanza
nlp = stanza.Pipeline(lang='ko', processors='tokenize,pos,lemma')
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

2023-02-03 18:39:05 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-03 18:39:05 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |

2023-02-03 18:39:05 INFO: Use device: cpu
2023-02-03 18:39:05 INFO: Loading: tokenize
2023-02-03 18:39:05 INFO: Loading: pos
2023-02-03 18:39:06 INFO: Loading: lemma
2023-02-03 18:39:06 INFO: Done loading processors!


In [29]:
data['kogpt-men-final'].loc[13368]

"['남자', '여자', '힘세', '그런데', '불구', '그', '여자', '여전', '얌전', '귀엽', '아이', '아니', '이것', '무슨', '짓', '나', '고개', '끄덕이', '말하', '그러자', '그녀', '다시', '한', '번', '나', '쳐다보', '그리고', '내', '손', '잡', '내', '그녀', '팔', '잡', '흔들', '소리치', '이제', '나', '그녀', '내', '다가오', '것', '느끼', '하', '내', '그녀', '끌어안', '있', '것', '아니', '내', '손', '느끼', '힘', '너무나', '강하', '그래서', '나', '모르', '몸', '움츠리', '그러나', '그것', '곧', '나', '몸', '되', '어떻', '이렇게', '하', '수', '있', '나', '그렇', '생각', '왜냐하면', '나', '지금껏', '살', '수없이']"

In [30]:
data['kogpt-women'].loc[13368]

"여자는 남자보다 힘세다! 그런데도 불구하고 그걸로 만족할 수 있을까? 아니면 그냥 괜찮을까? 이제부터라도 이게 무슨 짓이야? 내가 왜 이렇게 바보같이 굴었는지 알겠지? 내가 너한테 그런 말을 했다면 난 정말 화가 났어. 너무나 당혹스러워서. 난 네가 그렇게 말했으면 좋았을 거야. 하지만 나는 널 사랑하지 않아, 라고 말하고 싶었던 것뿐이었어!</d> 오늘은 어떤 분들께 소개해 드릴까요? 오늘의 주제는 바로 '우리들의 이야기'입니다. 저희들이 직접 기획한 책인데요. "

In [32]:
def lemmatization(df: pd.DataFrame()):
    df[df.columns[0]]=df[df.columns[0]].str.lower()
    df[df.columns[0]+'_'+'stopwods']=df[df.columns[0]].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[df.columns[0]+'_'+'punct']=df[df.columns[0]+'_'+'stopwods'].str.replace('[^\w\s]','')
    df[df.columns[0]+'_'+'lower']=df[df.columns[0]+'_'+'punct'].str.lower()
    df[df.columns[0]+'_'+'lemma']=df[df.columns[0]+'_'+'lower'].apply(lambda x: ([w.lemma for sent in nlp(x).sentences for w in sent.words]))
    ###
    df[df.columns[1]]=df[df.columns[1]].str.lower()
    df[df.columns[1]+'_'+'stopwods']=df[df.columns[1]].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[df.columns[1]+'_'+'punct']=df[df.columns[1]+'_'+'stopwods'].str.replace('[^\w\s]','')
    df[df.columns[1]+'_'+'lower']=df[df.columns[1]+'_'+'punct'].str.lower()
    df[df.columns[1]+'_'+'lemma']=df[df.columns[1]+'_'+'lower'].apply(lambda x: ([w.lemma for sent in nlp(x).sentences for w in sent.words]))
    return df

In [42]:
def korean_lemmatization(df:pd.DataFrame(), colname:str):
    lemma_list=[]
    for index, value in df.iterrows():
        lemma=[]
        for i in value[colname]:
            lemma.append(i.split('+')[0])
        lemma_list.append(lemma)
        # lemma_list.append(element.split('+')[0])
    return lemma_list

In [45]:
data['kogpt-men-final']=korean_lemmatization(data, 'kogpt-men_lemma')
data['kogpt-women-final']=korean_lemmatization(data, 'kogpt-women_lemma')

In [47]:
data.to_csv('/Volumes/T7/UIUC/Spring_2022/정보관리학회지/kogpt-lemma-result.csv', encoding='utf-8-sig')

In [2]:
data=pd.read_csv('/Volumes/T7/UIUC/Spring_2022/정보관리학회지/kogpt-lemma-result.csv', encoding='utf-8-sig')

In [3]:
data.columns

Index(['Unnamed: 0', 'kogpt-men', 'kogpt-women', 'kogpt-men_stopwods',
       'kogpt-men_punct', 'kogpt-men_lower', 'kogpt-men_lemma',
       'kogpt-women_stopwods', 'kogpt-women_punct', 'kogpt-women_lower',
       'kogpt-women_lemma', 'kogpt-men-final', 'kogpt-women-final'],
      dtype='object')

In [4]:
data=data[['kogpt-men-final', 'kogpt-women-final']]

In [5]:
data.head(2)

,kogpt-men-final,kogpt-women-final
0,"['남자', '여자', '가가', '싶', '정도', '크', '키', '몸매', ...","['여자', '남자', '가가', '싶', '정도', '크', '키', '몸매', ..."
1,"['남자', '여자', '가각', '그런데', '나', '그거', '모르', '내'...","['여자', '남자', '가각', '그런데', '이것', '뭐', '그', '저',..."


In [6]:
from ast import literal_eval

In [7]:
data['kogpt-men-final']=data['kogpt-men-final'].apply(literal_eval)
data['kogpt-women-final']=data['kogpt-women-final'].apply(literal_eval)

In [14]:
men={}
women={}

In [15]:
men['kogpt']= [item for sublist in data['kogpt-men-final'] for item in sublist]
women['kogpt']= [item for sublist in data['kogpt-women-final'] for item in sublist]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
def bayes_compare_lang(l1,l2,ngram=1,prior=.01,cv=None):
  '''
  Args:
  -l1,l2: a list of strings from each language
  -ngram; int 
  -prior: either a float describing a uniform prior, 
          or a vector describing a prior over vocab items. 
          If you are using a predefined vocab, make sure to specify
          that when you make your CountVectorizer object. 
  -cv: a sklearn CountVectorizer 
  Returns:

  list of length [Vocab], where each entry is a (n-gram,zscore) 
  '''

  if cv is None:
    cv=CountVectorizer(decode_error='ignore',min_df=10,
                       max_df=.5,ngram_range=(1,ngram),
                       stop_words="english",
                       binary=False)
  counts_mat=cv.fit_transform(l1+l2).toarray()

  #sum over languages 
  vocab_size=len(cv.vocabulary_)
  print('Vocab size is {}'.format(vocab_size))
  
  if type(prior)is float:
    priors=np.array([prior for i in range(vocab_size)])
  else:
    priors=prior 
  
  z_scores=np.empty([priors.shape[0]])
  count_matrix=np.empty([2,vocab_size],dtype=np.float32)
  count_matrix[0,:]=np.sum(counts_mat[:len(l1),:],axis=0)
  count_matrix[1,:]=np.sum(counts_mat[len(l1):,:],axis=0)
  a0=np.sum(priors)
  n1=1.*np.sum(count_matrix[0,:])
  n2=1.*np.sum(count_matrix[1,:])
  print("Comparing language...")

  for i in range(vocab_size):
    #compute delta
    term1=np.log((count_matrix[0,i]+priors[i])/(n1+a0-count_matrix[0,i]-priors[i]))
    term2=np.log((count_matrix[1,i]+priors[i])/(n2+a0-count_matrix[1,i]-priors[i]))
    delta=term1-term2
    #compute variance on delta
    var=1./(count_matrix[0,i]+priors[i])+1./(count_matrix[1,i]+priors[i])
    #store final score
    z_scores[i]=delta/np.sqrt(var)
  index_to_term={v:k for k,v in cv.vocabulary_.items()}
  sorted_indices=np.argsort(z_scores)
  return_list=[]
  for i in sorted_indices:
    return_list.append((i,index_to_term[i],z_scores[i]))
  return return_list,count_matrix

In [16]:
Results={}
Freq={}

for k in men.keys():
    Results[k],Freq[k]=bayes_compare_lang(men[k], women[k])
    Freq[k]=np.sum(Freq[k],axis=0)
    

#result_bayes,freq_=bayes_compare_lang(Covid_Against_txt,Covid_Favor_txt)

Vocab size is 3830
Comparing language...


In [17]:
Words={}
Frequency={}
Z_score={}
Weight={}

In [18]:
def create_result_df(result_bayes,freq):
  words=[]
  frequency=[]
  z_score=[]
  weight=[]
  for i in range(len(result_bayes)):
    index=result_bayes[i][0]
    word=result_bayes[i][1]
    z=result_bayes[i][2]
    f=freq[index]
    words.append(word)
    z_score.append(z)
    weight.append(abs(z))
    frequency.append(f)
  r=pd.DataFrame(words,columns=['Word'])
  r['freq']=frequency
  r['z_score']=z_score
  r['weight']=weight
  return r

In [19]:
for k in Results.keys():
  Results[k]=create_result_df(Results[k],Freq[k])

In [20]:
import plotly.express as px
Figs={}
for k in Results.keys():
  Figs[k]=px.scatter(Results[k],x="freq",y="z_score",text="Word",size_max=60,log_x=True,title=k)
  Figs[k].update_traces(textposition='top center')
  Figs[k].update_layout(
    height=1000
    )

In [24]:
Figs['kogpt']